# LSTM으로 텍스트 생성하기

## 글자 수준의 LSTM 텍스트 생성 모델 구현

이런 아이디어를 케라스로 구현해 보죠. 먼저 언어 모델을 학습하기 위해 많은 텍스트 데이터가 필요합니다. 위키피디아나 반지의 제왕처럼 아주 큰 텍스트 파일이나 텍스트 파일의 묶음을 사용할 수 있습니다. 이 예에서는 19세기 후반 독일의 철학자 니체의 글을 사용하겠습니다(영어로 번역된 글입니다). 학습할 언어 모델은 일반적인 영어 모델이 아니라 니체의 문체와 특정 주제를 따르는 모델일 것입니다.


In [1]:
from tensorflow.keras import utils
import numpy as np

path = utils.get_file(
    'nietzche.txt',
    origin = 'https://s3.amazonaws.com/text-datasets/nietzsche.txt')

text = open(path).read().lower()
print('말뭉치 크기:', len(text))


614400/600901 [==============================] - 0s 0us/step
말뭉치 크기: 600893


In [2]:
text[:500]

'preface\n\n\nsupposing that truth is a woman--what then? is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to truth, have been unskilled and unseemly methods for\nwinning a woman? certainly she has never allowed herself to be won; and\nat present every kind of dogma stands with sad and discouraged mien--if,\nindeed, it s'

## 60개 글자로 된 시퀀스를 추출

In [3]:
maxlen = 60 # 60개 글자로 된 시퀀스 추출
step = 3 # 세 글자씩 건너뛰면서 새로운 시퀀스를 샘플링

sentences = [] # 추출한 시퀀스를 담을 리스트
next_chars = [] # 타깃(시퀀스 다음 글자)을 담을 리스트

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])
print('시퀀스 개수:', len(sentences))

chars = sorted(list(set(text))) # 말뭉치에서 고유한 글자를 담은 리스트
print('고유한 글자:', len(chars))

char_indices = dict((char, char.index(char)) for char in chars) # chars 리스트에 있는 글자와 글자의 인덱스를 매핑한 딕셔너리

시퀀스 개수: 200278
고유한 글자: 57


In [4]:
maxlen = 60

# 세글자씩 건너 뛰면서 새로운 시퀀스를 생성
step = 3

sentences = [] # 추출한 시퀀스를 담을 리스트
next_chars = [] # 타깃(시퀀스 다음글자)을 담을 리스트

for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i: i+maxlen])
  next_chars.append(text[i + maxlen])
  print('시퀀스 개수:', len(sentences))


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
시퀀스 개수: 195279
시퀀스 개수: 195280
시퀀스 개수: 195281
시퀀스 개수: 195282
시퀀스 개수: 195283
시퀀스 개수: 195284
시퀀스 개수: 195285
시퀀스 개수: 195286
시퀀스 개수: 195287
시퀀스 개수: 195288
시퀀스 개수: 195289
시퀀스 개수: 195290
시퀀스 개수: 195291
시퀀스 개수: 195292
시퀀스 개수: 195293
시퀀스 개수: 195294
시퀀스 개수: 195295
시퀀스 개수: 195296
시퀀스 개수: 195297
시퀀스 개수: 195298
시퀀스 개수: 195299
시퀀스 개수: 195300
시퀀스 개수: 195301
시퀀스 개수: 195302
시퀀스 개수: 195303
시퀀스 개수: 195304
시퀀스 개수: 195305
시퀀스 개수: 195306
시퀀스 개수: 195307
시퀀스 개수: 195308
시퀀스 개수: 195309
시퀀스 개수: 195310
시퀀스 개수: 195311
시퀀스 개수: 195312
시퀀스 개수: 195313
시퀀스 개수: 195314
시퀀스 개수: 195315
시퀀스 개수: 195316
시퀀스 개수: 195317
시퀀스 개수: 195318
시퀀스 개수: 195319
시퀀스 개수: 195320
시퀀스 개수: 195321
시퀀스 개수: 195322
시퀀스 개수: 195323
시퀀스 개수: 195324
시퀀스 개수: 195325
시퀀스 개수: 195326
시퀀스 개수: 195327
시퀀스 개수: 195328
시퀀스 개수: 195329
시퀀스 개수: 195330
시퀀스 개수: 195331
시퀀스 개수: 195332
시퀀스 개수: 195333
시퀀스 개수: 195334
시퀀스 개수: 195335
시퀀스 개수: 195336
시퀀스 개수: 195337
시퀀스 개수: 195338
시퀀스 개수: 195339
시퀀스 개수: 195340
시퀀스 개수: 195341
시퀀스 개수: 195342
시퀀스 

## 말뭉치에서 고유한 글자를 담은 리스트 처리

In [5]:
chars = sorted(list(set(text)))
print('고유한 글자:', len(chars))
 

고유한 글자: 57


## chars 리스트에 있는 글자와 글자의 인덱스를 매핑한 딕셔너리

In [6]:
char_indices = dict((char, chars.index(char)) for char in chars)

print('벡터화...')

벡터화...


## 글자를 원-핫 인코딩하여 0과 1의 이진 배열로 변경

In [7]:
x = np.zeros((len(sentences), maxlen, len(chars)),dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
   for t,char in enumerate(sentence):
       x[i, t, char_indices[char]] = 1
   y[i, char_indices[next_chars[i]]] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


## 네트워크 구성

In [8]:
from tensorflow.keras import layers,models

model = models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [9]:
from tensorflow.keras import optimizers

optimizer = optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


## 언어 모델 훈련과 샘플링

훈련된 모델과 시드로 쓰일 간단한 텍스트가 주어지면 다음과 같이 반복하여 새로운 텍스트를 생성할 수 있습니다.

1.	지금까지 생성된 텍스트를 주입하여 모델에서 다음 글자에 대한 확률 분포를 뽑습니다.
2.	특정 온도로 이 확률 분포의 가중치를 조정합니다.
3.	가중치가 조정된 분포에서 무작위로 새로운 글자를 샘플링합니다.
4.	새로운 글자를 생성된 텍스트의 끝에 추가합니다.

다음 코드는 모델에서 나온 원본 확률 분포의 가중치를 조정하고 새로운 글자의 인덱스를 추출합니다(샘플링 함수입니다):


밑이 자연상수 e인 지수함수(e^x)의 그래프<br>
https://wooono.tistory.com/214

Python Numpy.log()-로그 <br>
https://www.delftstack.com/ko/api/numpy/python-numpy-log/


In [10]:
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

[과제] 텍스트를 생성한 후 temperature에 따라 생성된 text가 어떻게 다른지 비교하세요

In [11]:
import random
import sys

random.seed(42)
# 데이터에서 한 번만 반복해서 모델을 학습합니다.
start_index = random.randint(0, len(text) - maxlen - 1)

for epoch in range(1, 60):
     print('에포크',epoch)
model.fit(x,y, batch_size=128, epochs=1)

# 무작위로 시드 텍스트를 선택합니다
seed_text = text[start_index : start_index + maxlen]
print('--- 시드 텍스트:' +seed_text + '"')

# 여러가지 샘플링 온도를 시도합니다.
for temperature in [0.2,0.5,1.0,1.2]:
  print('---- 온도:',temperature)
  generated_text = seed_text
  sys.stdout.write(generated_text)

  for i in range(400):
    sampled = np.zeros((1,maxlen, len(chars)))
    for t, char in enumerate(generated_text):
      sampled[0, t, char_indices[char]] = 1.

    preds = model.predict(sampled, verbose=0)[0]
    next_index = sample(preds, temperature)
    next_char = chars[next_index]
    
    generated_text += next_char
    generated_text = generated_text[1:]

    sys.stdout.write(next_char)
    sys.stdout.flush()
  print()

에포크 1
에포크 2
에포크 3
에포크 4
에포크 5
에포크 6
에포크 7
에포크 8
에포크 9
에포크 10
에포크 11
에포크 12
에포크 13
에포크 14
에포크 15
에포크 16
에포크 17
에포크 18
에포크 19
에포크 20
에포크 21
에포크 22
에포크 23
에포크 24
에포크 25
에포크 26
에포크 27
에포크 28
에포크 29
에포크 30
에포크 31
에포크 32
에포크 33
에포크 34
에포크 35
에포크 36
에포크 37
에포크 38
에포크 39
에포크 40
에포크 41
에포크 42
에포크 43
에포크 44
에포크 45
에포크 46
에포크 47
에포크 48
에포크 49
에포크 50
에포크 51
에포크 52
에포크 53
에포크 54
에포크 55
에포크 56
에포크 57
에포크 58
에포크 59
1565/1565 [==============================] - 166s 103ms/step - loss: 1.9753
--- 시드 텍스트:the slowly ascending ranks and classes, in which,
through fo"
---- 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the strong and the strong to the strong the strong the specially the strong of the strong the strong the man in the strong and the indeed to the stranger and should the strong the greated in the realy the strong the greated the strong to the sting the strong the consection of the strong the still to the strong the strong the strong the strong the strong the strong the c